# Process the Spectrum of the Host Star

In [2]:
#import the libraries 

import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import fftpack,signal
import matplotlib
from matplotlib.ticker import ScalarFormatter
from astropy.visualization import (MinMaxInterval, SqrtStretch, ImageNormalize, ZScaleInterval)
import scipy
import json
import subprocess
import os

from exocrires import spectra_2d
from exocrires import analysis

In [ ]:
data=spectra_2d.load_json('/Volumes/JCP_work/thesis_ms/input_1709_AO.json','r')

print(data['target']['brightness']['params']['mag'])


In [ ]:
out_put=[]
for i in data['output']:
    if i == 'psf':
        out_put.append('%s'%i)
    else:   
        for j in data['output'][i]:
            out_put.append('%s_%s'%(i,j))
out_put=[string.lower() for string in out_put] 

In [ ]:
simu_star=spectra_2d.spectra_2d(path_etc_local='/Volumes/JCP_work/thesis_ms/etc_cli.py', path_etc_input='/Volumes/JCP_work/thesis_ms/input_1709_AO.json', path_input_modi='/Volumes/JCP_work/thesis_ms/modi_input_1709_AO.json',\
             path_etc_output='/Volumes/JCP_work/thesis_ms/2411/star/', target='star', aperture_size=35, date='2411',\
        order_window=[[1921.318, 1961.128],[1989.978, 2031.165],[2063.711, 2106.392],[2143.087, 2187.386], [2228.786, 2274.835],\
        [2321.596, 2369.534], [2422.415, 2472.388]],\
        order_number=[29, 28, 27, 26, 25, 24, 23])

simu_star.output_json()

simu_star.json2ascii(path_json2ascii='/Volumes/JCP_work/thesis_ms/etc_json2ascii.py')

simu_star.ascii2txt(output=out_put)

data_log=simu_star.signal(detectors=3, contribution='target', focal_plane=(2048, 2048*3))

data_log.to_csv(simu_star.path_output+'stellar_simu.csv')

In [ ]:
simu_star.plot_signal(wave_count_sort=data_log, max_percentile=99,\
                      save_path=simu_star.path_output+'star_simu.png', interv=ZScaleInterval())

# Process the Sky Background

In [ ]:
# The sky background distribution should be the same for each run of output data. Therefore we only need to run it once with any input specgtrum of the planet

path='/Volumes/JCP_work/thesis_ms'

data_log_k=simu_star.signal(detectors=3, focal_plane=(2048, 2048*3), contribution='sky')
data_log_k.to_csv(path+'/2411/sky_noi/sky_simu.csv')
simu_star.plot_signal(data_log_k, max_percentile=99.99, save_path=path+'/2411/sky_noi/sky_simu.png', interv=ZScaleInterval(), Nor=None)

# Process the Spectrum of the Planet and Make Combination

In [7]:
#set up the array for contrast data
cont=np.zeros((55, 2, 2046*3))

#set up noise parameters
read=6 #e-/pix
dark=0.003 #e-/pix/s
dit=120
ndit=18
nspec=1
nspat=119

npix=nspec*nspat
ndark=dit*dark #e-/pix/exposure

#set up the angular separation (pix)
po=int(np.ceil(0.319/0.059))

In [ ]:
data_pp=spectra_2d.load_json('/Volumes/JCP_work/thesis_ms/input_1709_p_AO.json', 'r')

print(data_pp['target']['brightness']['params']['mag'])

In [ ]:
out_put=[]
for i in data_pp['output']:
    if i == 'psf':
        out_put.append('%s'%i)
    else:   
        for j in data_pp['output'][i]:
            out_put.append('%s_%s'%(i,j))
out_put=[string.lower() for string in out_put] 

In [ ]:
path='/Volumes/JCP_work/thesis_ms'

filenames=glob.glob(path+'/models_1683734698/bt-settl_m/lte*.dat') #use dat
filenames=sorted(filenames)

for filename in filenames[0:2]:
    
    #if os.path.exists(path+'/2411/planet/%s'%filename[-34:-23]) == True:
    #continue
    
    simu_planet=spectra_2d.spectra_2d(path_etc_local=path+'/etc_cli.py', path_etc_input=path+'/input_1709_p_AO.json', path_input_modi=path+'/modi_input_1709_p_AO.json',\
                 path_etc_output=path+'/2411/planet/%s/'%filename[-34:-23], target='planet', aperture_size=35, date='2411',\
            order_window=[[1921.318, 1961.128],[1989.978, 2031.165],[2063.711, 2106.392],[2143.087, 2187.386], [2228.786, 2274.835],\
            [2321.596, 2369.534], [2422.415, 2472.388]],\
            order_number=[29, 28, 27, 26, 25, 24, 23],\
            model_name=filename)

    simu_planet.output_json()

    simu_planet.json2ascii(path_json2ascii=path+'/etc_json2ascii.py')

    simu_planet.ascii2txt(output=out_put)

    data_log_p=simu_planet.signal(detectors=3, focal_plane=(2048, 2048*3), contribution='target')

    data_log_p.to_csv(simu_planet.path_output+'planet_simu.csv')
    
    print ('Planet-only 2d spectrum done:%s'%simu_planet.path_output)
    
    simu_planet.plot_signal(data_log_p, max_percentile=99,\
                            save_path=simu_planet.path_output+'planet_simu.png', interv=ZScaleInterval())
    
    ##-----Combine with the sky background, the stellar light and the noise terms-----##
    
    tot_data_com, fig=simu_planet.combine(data_star=data_log, data_sky=data_log_k, data_p=data_log_p, ron=read, dark=ndark, d=po, \
                                focal_plane=(2048,2048*3), plot_combination=True)
    fig.savefig(path+'/2411/planet/%s/combine.png'%filename[-34:-23])

    np.save(path+'/2411/planet/%s/combine_focal_plane.npy'%filename[-34:-23], tot_data_com)

In [9]:
#plot the contrast and save the average value
path='/Volumes/JCP_work/thesis_ms'

filenames=glob.glob(path+'/models_1683734698/bt-settl_m/lte*.dat') #use dat
filenames=sorted(filenames)

data_log=pd.read_csv(path+'/2411/star/stellar_simu.csv')

i=0

for filename in filenames[0:2]:
    
    data_log_p=pd.read_csv(path+'/2411/planet/%s/planet_simu.csv'%filename[-34:-23])
    
    p=analysis.planet_para(disper_planet=data_log_p, disper_star=data_log)
    p2s_in, p2s_pl, plot_in, plot_pl=p.p_2_s(order_num=[29, 28, 27, 26, 25, 24, 23], planet_posi=po)
    
    plot_in.savefig(path+'/2411/planet/%s/contrast_intrinsic.png'%filename[-34:-23], dpi=200)
    plot_pl.savefig(path+'/2411/planet/%s/contrast_planet_posi.png'%filename[-34:-23], dpi=200)
    np.save(path+'/2411/planet/%s/contrast.npy'%filename[-34:-23], np.array((p2s_in, p2s_pl)))
    
    a=p2s_in[2046*12:2046*15]
    b=p2s_pl[2046*12:2046*15]
    cont[i]=np.array((a,b))
    i+=1
    
    print ('Combination done;%s'%filename[-34:-23])

/Users/richard/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


Combination done;010-4.0-0.0


/Users/richard/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


Combination done;010-4.5-0.0
